<a href="https://colab.research.google.com/github/fphans/BIDS-NGS/blob/main/NGS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
install.packages("googledrive")
library("googledrive")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
###################
# RNA Seq Analyse #
# Datum: 18.06.2021
# Autor: Patrick Metzger

################
# Introduction #

# Wir bitten Sie hier, die grundlegenden Schritte einer RNA-Sequenzierungs-Analyse durchzufÃ¼hren.
# Wir stellen Ihnen bereits die zum menschlichen Referenzgenom zugeordneten (aligned) Reads zur VerfÃ¼gung,
# d.h. Ihr Ausgangspunkt sind die âCountsâ fÃ¼r jede Probe. Zusammen mit den Counts (*_ReadsPerGene.out.tab) stellen wir Ihnen die
# Annotationstabelle (targets.xlsx) zur VerfÃ¼gung, die die Proben-IDs mit den tatsÃ¤chlichen biologischen Bedingungen und
# "Real-World-Proben" verknÃ¼pft. Der Datensatz stammt aus einem realen wissenschaftlichen Projekt, an dem
# wir hier in Freiburg gearbeitet haben. Die DatensÃ¤tze enthalten Counts von einer einzelnen Zelllinie, nÃ¤mlich MiaPaCa2 (Pankreas),
# und zwei biologischen Bedingungen. Das Ziel der Studie ist es, den Einfluss einer KLF7-Ãberexpression zu identifizieren.
# Dazu wurde die Zelllinie so modifiziert, dass sie KLF7 Ã¼berexprimiert. Als Kontrolle wurde die unbehandelte Zelllinie verwendet.
# Die Ã¼berexprimierende Bedingung ist mit "*_KLF7" und die Kontrolle mit "*_LV" beschriftet. LV steht hier fÃ¼r "Leervektor".
# Ihre Aufgabe ist es nun, die signifikant verÃ¤nderten Gene (in Bezug auf die Genexpression) zwischen den behandelten Proben (KLF7)
# und der unbehandelten Kontrolle (LV) zu identifizieren. Dazu muss eine Analyse der differentiell exprimierten Gene
# (diffenretially expressed genes DEG) durchgefÃ¼hrt werden. Im folgenden Skript geben wir Ihnen dazu einige Hinweise und Hilfestellungen.

# Die folgenden Schritte werden abgedeckt

# 1. die ZÃ¤hldaten mÃ¼ssen importiert werden
# 2. die Proben mit den biologischen Bedingungen verknÃ¼pfen
# 3. eine Meta-Tabelle mit allen gemessenen Genen und allen verfÃ¼gbaren Gen-Identifikatoren erzeugen
# 4. eine Hauptkomponentenanalyse (PCA) berechnen, um einen ersten Ãberblick Ã¼ber den Datensatz zu erhalten
# 5. fÃ¼hren Sie die eigentliche DEG-Analyse durch. Wie viele Gene sind signifikant verÃ¤ndert bis zu einem korrigiertem p-Wert < 0,05; Wie viele davon sind hoch- bzw. herunterreguliert?
# 6. visualisieren und exportieren Sie Ihre Ergebnisse
# 7. erzeugen Sie Heatmaps der hochvariablen Gene


#####################
# NÃ¼tzliche Befehle #

# head()
# ?functionName or help()
# install.packages()
# accessing columns by name, e.g. result$padj
# filter data.frame, e.g. res[res$test > 5,] the output contains all columns of res but only the rows fulfilling that the value within the "test" column contains a value bigger than 5
# dim()
# length()
# is.na() / !is.na()

###################
# NÃ¼tzliche Links #

# links / tutorials / vignettes
# https://www.bioconductor.org/install/
# https://bioconductor.org/packages/release/bioc/html/DESeq2.html
# https://cran.r-project.org/web/packages/FactoMineR/index.html
# https://ggplot2.tidyverse.org ; https://cran.r-project.org/web/packages/ggplot2/index.html
# https://cran.r-project.org/web/packages/ggrepel/vignettes/ggrepel.html
# https://bioconductor.org/packages/release/bioc/html/genefilter.html

#############
# Libraries #

# used packages
# if not installed please install with either bioconductor BiocManager::install() (see useful links) or install.packages()

if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
BiocManager::install(version = "3.13")
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
#BiocManager::install("Pakete")
#BiocManager::install("DESeq2")
#BiocManager::install(c('pheatmap','openxlsx','org.Hs.eg.db','ggplot2','genefilter','FactoMineR','ggrepel','apeglm','ggrepel','genefilter'))

library(DESeq2)
library(openxlsx)
library(pheatmap)
library(org.Hs.eg.db)
library(ggplot2)
library(genefilter)
library(FactoMineR)
library(ggrepel)
library(apeglm)
library(ggrepel)
library(genefilter)

# Define Working Directory
setwd("~/R/bi") # change according to were you put the data

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com


Bioconductor version 3.13 (BiocManager 1.30.16), R 4.1.0 (2021-05-18)

Installing package(s) 'BiocVersion'

Old packages: 'broom', 'colorspace', 'curl', 'dplyr', 'gert', 'ggplot2',
  'mime', 'openssl', 'rmarkdown', 'testthat', 'xfun'



ERROR: ignored